# Data Preprocessing / Exploratory Data Analysis

## Initial Preprocessing
1. Handling of NaN values and duplicates
2. Dropping unnecessary columns
3. Dropping rows where OBJECT_TYPE is unknown
4. One Hot Encoding of Categorical features

In [2]:
import pandas as pd

In [150]:
df = pd.read_csv("data/space_debris_kaggle.csv")
df.head()

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T06:46:11,18 SPCS,ARIANE 42P+ DEB,1992-072J,EARTH,TEME,UTC,SGP4,...,MEDIUM,FR,1992.0,FRGUI,NaN,3195178,188614016,0 ARIANE 42P+ DEB,1 26741U 92072J 21304.94919376 .00000883 0...,2 26741 7.7156 90.2410 6528926 243.1216 38...
1,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T04:58:37,18 SPCS,SL-8 DEB,1979-028C,EARTH,TEME,UTC,SGP4,...,SMALL,CIS,1979.0,PKMTR,NaN,3194950,188593285,0 SL-8 DEB,1 26743U 79028C 21304.68908982 .00000079 0...,2 26743 82.9193 299.1120 0030720 158.9093 201...
2,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T06:26:11,18 SPCS,GSAT 1,2001-015A,EARTH,TEME,UTC,SGP4,...,LARGE,IND,2001.0,SRI,NaN,3195026,188609573,0 GSAT 1,1 26745U 01015A 21305.22411368 -.00000165 0...,2 26745 12.1717 16.5368 0237386 250.1248 146...
3,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-31T18:07:15,18 SPCS,CZ-4 DEB,1999-057MB,EARTH,TEME,UTC,SGP4,...,SMALL,PRC,1999.0,TSC,NaN,3194431,188556894,0 CZ-4 DEB,1 26754U 99057MB 21304.46625230 .00002265 0...,2 26754 98.4781 8.7205 0060618 37.3771 323...
4,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T04:58:37,18 SPCS,CZ-4 DEB,1999-057MC,EARTH,TEME,UTC,SGP4,...,SMALL,PRC,1999.0,TSC,NaN,3194950,188592541,0 CZ-4 DEB,1 26755U 99057MC 21304.74081807 .00002610 0...,2 26755 98.4232 122.0724 0062255 345.1605 27...


In [151]:
df.columns

Index(['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR',
       'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM',
       'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS',
       'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE',
       'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2'],
      dtype='object')

In [152]:
#Drop columns with only one unique value
cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]

# Drop these columns from the DataFrame
df.drop(columns=cols_to_drop, inplace=True)

# Optionally, check if the columns are dropped
print("Dropped columns:", cols_to_drop)

Dropped columns: ['CCSDS_OMM_VERS', 'COMMENT', 'ORIGINATOR', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'ELEMENT_SET_NO']


In [153]:
print(df.isnull().sum())

CREATION_DATE            0
OBJECT_NAME              0
OBJECT_ID               39
EPOCH                    0
MEAN_MOTION              0
ECCENTRICITY             0
INCLINATION              0
RA_OF_ASC_NODE           0
ARG_OF_PERICENTER        0
MEAN_ANOMALY             0
NORAD_CAT_ID             0
REV_AT_EPOCH             0
BSTAR                    0
MEAN_MOTION_DOT          0
MEAN_MOTION_DDOT         0
SEMIMAJOR_AXIS           0
PERIOD                   0
APOAPSIS                 0
PERIAPSIS                0
OBJECT_TYPE              0
RCS_SIZE               198
COUNTRY_CODE            39
LAUNCH_DATE             39
SITE                    39
DECAY_DATE           14372
FILE                     0
GP_ID                    0
TLE_LINE0                0
TLE_LINE1                0
TLE_LINE2                0
dtype: int64


In [137]:
print(df.duplicated().sum())

0


In [138]:
df['OBJECT_TYPE'].value_counts()

OBJECT_TYPE
DEBRIS         8431
PAYLOAD        4950
ROCKET BODY     744
TBA             247
Name: count, dtype: int64

In [154]:
#Dropping DECAY_DATE as all the columns are Nan
df.drop(columns=['DECAY_DATE'], inplace=True)

In [155]:
# Handle missing OBJECT_ID based on OBJECT_TYPE
df['OBJECT_ID'].fillna('Unknown', inplace=True)

/tmp/ipykernel_9342/704097546.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['OBJECT_ID'].fillna('Unknown', inplace=True)


In [156]:
df['RCS_SIZE'].value_counts()

RCS_SIZE
SMALL     8346
LARGE     4189
MEDIUM    1639
Name: count, dtype: int64

In [157]:
#Removing rows where OBJECT_TYPE is TBA
df = df[df['OBJECT_TYPE'] != 'TBA']

In [158]:
df['COUNTRY_CODE'].fillna(df['COUNTRY_CODE'].mode()[0], inplace=True)
df['LAUNCH_DATE'].fillna(df['LAUNCH_DATE'].mode()[0], inplace=True)
df['SITE'].fillna(df['SITE'].mode()[0], inplace=True)

/tmp/ipykernel_9342/2924190457.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['COUNTRY_CODE'].fillna(df['COUNTRY_CODE'].mode()[0], inplace=True)
/tmp/ipykernel_9342/2924190457.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value,

In [159]:
df['RCS_SIZE'].fillna('UNKNOWN', inplace=True)
print(df['RCS_SIZE'].isnull().sum())

0


/tmp/ipykernel_9342/1611599753.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['RCS_SIZE'].fillna('UNKNOWN', inplace=True)


In [161]:
df['RCS_SIZE'].value_counts()

RCS_SIZE
SMALL      8256
LARGE      4170
MEDIUM     1540
UNKNOWN     159
Name: count, dtype: int64

In [162]:
#One Hot Encode RCS_SIZE
from sklearn.preprocessing import OneHotEncoder

# One Hot Encode RCS_SIZE
encoder = OneHotEncoder(sparse_output=False)

# Reshape the 'RCS_SIZE' column to 2D
rcs_size_reshaped = df[['RCS_SIZE']]

# Fit transform
rcs_size_encoded = encoder.fit_transform(rcs_size_reshaped)

# Convert to DataFrame and ensure indices match the original DataFrame
rcs_size_encoded_df = pd.DataFrame(rcs_size_encoded,
                                   columns=encoder.get_feature_names_out(['RCS_SIZE']),
                                   index=df.index)  # Match indices

# Concatenate the original dataframe with the new one-hot encoded columns
df = pd.concat([df, rcs_size_encoded_df], axis=1)


In [163]:
print(df.isnull().sum())

CREATION_DATE        0
OBJECT_NAME          0
OBJECT_ID            0
EPOCH                0
MEAN_MOTION          0
ECCENTRICITY         0
INCLINATION          0
RA_OF_ASC_NODE       0
ARG_OF_PERICENTER    0
MEAN_ANOMALY         0
NORAD_CAT_ID         0
REV_AT_EPOCH         0
BSTAR                0
MEAN_MOTION_DOT      0
MEAN_MOTION_DDOT     0
SEMIMAJOR_AXIS       0
PERIOD               0
APOAPSIS             0
PERIAPSIS            0
OBJECT_TYPE          0
RCS_SIZE             0
COUNTRY_CODE         0
LAUNCH_DATE          0
SITE                 0
FILE                 0
GP_ID                0
TLE_LINE0            0
TLE_LINE1            0
TLE_LINE2            0
RCS_SIZE_LARGE       0
RCS_SIZE_MEDIUM      0
RCS_SIZE_SMALL       0
RCS_SIZE_UNKNOWN     0
dtype: int64


## Splitting of Features and Target
1. Label Encoding Target (OBJECT_TYPE)
2. Splitting into X and Y

In [164]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Encode OBJECT_TYPE
df['OBJECT_TYPE_ENCODED'] = label_encoder.fit_transform(df['OBJECT_TYPE'])

# Split into X and Y
X = df.drop(columns=['OBJECT_TYPE', 'OBJECT_TYPE_ENCODED'])  # Features
Y = df['OBJECT_TYPE_ENCODED']  # Encoded target

In [165]:
X

,CREATION_DATE,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,...,SITE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2,RCS_SIZE_LARGE,RCS_SIZE_MEDIUM,RCS_SIZE_SMALL,RCS_SIZE_UNKNOWN
0,2021-11-01T06:46:11,ARIANE 42P+ DEB,1992-072J,2021-10-31T22:46:50.340864,2.921700,0.652893,7.7156,90.2410,243.1216,38.7796,...,FRGUI,3195178,188614016,0 ARIANE 42P+ DEB,1 26741U 92072J 21304.94919376 .00000883 0...,2 26741 7.7156 90.2410 6528926 243.1216 38...,0.0,1.0,0.0,0.0
1,2021-11-01T04:58:37,SL-8 DEB,1979-028C,2021-10-31T16:32:17.360448,13.754973,0.003072,82.9193,299.1120,158.9093,201.3337,...,PKMTR,3194950,188593285,0 SL-8 DEB,1 26743U 79028C 21304.68908982 .00000079 0...,2 26743 82.9193 299.1120 0030720 158.9093 201...,0.0,0.0,1.0,0.0
2,2021-11-01T06:26:11,GSAT 1,2001-015A,2021-11-01T05:22:43.421952,1.038224,0.023739,12.1717,16.5368,250.1248,146.2900,...,SRI,3195026,188609573,0 GSAT 1,1 26745U 01015A 21305.22411368 -.00000165 0...,2 26745 12.1717 16.5368 0237386 250.1248 146...,1.0,0.0,0.0,0.0
3,2021-10-31T18:07:15,CZ-4 DEB,1999-057MB,2021-10-31T11:11:24.198720,14.775907,0.006062,98.4781,8.7205,37.3771,323.1632,...,TSC,3194431,188556894,0 CZ-4 DEB,1 26754U 99057MB 21304.46625230 .00002265 0...,2 26754 98.4781 8.7205 0060618 37.3771 323...,0.0,0.0,1.0,0.0
4,2021-11-01T04:58:37,CZ-4 DEB,1999-057MC,2021-10-31T17:46:46.681248,14.724482,0.006226,98.4232,122.0724,345.1605,27.6061,...,TSC,3194950,188592541,0 CZ-4 DEB,1 26755U 99057MC 21304.74081807 .00002610 0...,2 26755 98.4232 122.0724 0062255 345.1605 27...,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14328,2021-11-01T07:06:15,ATLAS 5 CENTAUR DEB,2018-079GA,2021-10-31T19:19:34.028544,1.947986,0.486499,11.1000,190.7386,42.3158,346.4724,...,AFETR,3195402,188619219,0 ATLAS 5 CENTAUR DEB,1 49373U 18079GA 21304.80525496 -.00000088 0...,2 49373 11.1000 190.7386 4864989 42.3158 346...,0.0,0.0,0.0,1.0
14329,2021-11-01T04:58:37,ATLAS 5 CENTAUR DEB,2018-079GC,2021-10-31T14:28:28.285824,1.726826,0.504225,11.6205,188.7505,18.9358,354.6612,...,AFETR,3194950,188608542,0 ATLAS 5 CENTAUR DEB,1 49375U 18079GC 21304.60310516 -.00000121 0...,2 49375 11.6205 188.7505 5042252 18.9358 354...,0.0,0.0,0.0,1.0
14330,2021-11-01T07:06:15,ATLAS 5 CENTAUR DEB,2018-079GD,2021-10-31T19:22:37.367616,1.648809,0.453078,10.8285,227.9659,324.3414,277.0792,...,AFETR,3195401,188618903,0 ATLAS 5 CENTAUR DEB,1 49376U 18079GD 21304.80737694 -.00000100 0...,2 49376 10.8285 227.9659 4530780 324.3414 277...,0.0,0.0,0.0,1.0
14331,2021-10-30T06:56:14,ATLAS 5 CENTAUR DEB,2018-079GF,2021-10-29T18:51:17.863488,1.610644,0.483557,10.7227,225.4025,321.7872,12.3257,...,AFETR,3192054,188474725,0 ATLAS 5 CENTAUR DEB,1 49378U 18079GF 21302.78562342 -.00000113 0...,2 49378 10.7227 225.4025 4835565 321.7872 12...,0.0,0.0,0.0,1.0


## Next Steps
Further Preprocessing on X:
1. Scale numeric features as needed
2. Feature Engineering and Data Visualization
3. Train Test Split to prepare for ML modeling